In [18]:
import torch
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
import ltn

## Data Preparation

In [19]:
!poetry run poe download-datasets

Poe => mkdir -p examples/datasets
Poe => curl -L -o examples/datasets/iris_training.csv https://raw.githubusercontent.com/tommasocarraro/LTNtorch/main/examples/datasets/iris_training.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2218  100  2218    0     0  64918      0 --:--:-- --:--:-- --:--:-- 65235
Poe => curl -L -o examples/datasets/iris_test.csv https://raw.githubusercontent.com/tommasocarraro/LTNtorch/main/examples/datasets/iris_test.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   598  100   598    0     0  18233      0 --:--:-- --:--:-- --:--:-- 18687


In [5]:
train_data = pd.read_csv("datasets/iris_training.csv")
test_data = pd.read_csv("datasets/iris_test.csv")

train_labels = train_data.pop("species")
test_labels = test_data.pop("species")

train_data = torch.tensor(train_data.to_numpy()).float()
test_data = torch.tensor(test_data.to_numpy()).float()
train_labels = torch.tensor(train_labels.to_numpy()).long()
test_labels = torch.tensor(test_labels.to_numpy()).long()

In [6]:
# we define predicate P
class MLP(torch.nn.Module):
    """
    This model returns the logits for the classes given an input example. It does not compute the softmax, so the output
    are not normalized.
    This is done to separate the accuracy computation from the satisfaction level computation. Go through the example
    to understand it.
    """
    def __init__(self, layer_sizes=(4, 16, 16, 8, 3)):
        super(MLP, self).__init__()
        self.elu = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(0.2)
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                                                  for i in range(1, len(layer_sizes))])

    def forward(self, x, training=False):
        """
        Method which defines the forward phase of the neural network for our multi class classification task.
        In particular, it returns the logits for the classes given an input example.

        :param x: the features of the example
        :param training: whether the network is in training mode (dropout applied) or validation mode (dropout not applied)
        :return: logits for example x
        """
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
            if training:
                x = self.dropout(x)
        logits = self.linear_layers[-1](x)
        return logits


class LogitsToPredicate(torch.nn.Module):
    """
    This model has inside a logits model, that is a model which compute logits for the classes given an input example x.
    The idea of this model is to keep logits and probabilities separated. The logits model returns the logits for an example,
    while this model returns the probabilities given the logits model.

    In particular, it takes as input an example x and a class label l. It applies the logits model to x to get the logits.
    Then, it applies a softmax function to get the probabilities per classes. Finally, it returns only the probability related
    to the given class l.
    """
    def __init__(self, logits_model):
        super(LogitsToPredicate, self).__init__()
        self.logits_model = logits_model
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, l, training=False):
        logits = self.logits_model(x, training=training)
        probs = self.softmax(logits)
        out = torch.sum(probs * l, dim=1)
        return out

In [7]:
class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        n = self.data.shape[0]
        idxlist = list(range(n))
        if self.shuffle:
            np.random.shuffle(idxlist)

        for _, start_idx in enumerate(range(0, n, self.batch_size)):
            end_idx = min(start_idx + self.batch_size, n)
            data = self.data[idxlist[start_idx:end_idx]]
            labels = self.labels[idxlist[start_idx:end_idx]]

            yield data, labels

# create train and test loader
train_loader = DataLoader(train_data, train_labels, 64, shuffle=True)
test_loader = DataLoader(test_data, test_labels, 64, shuffle=False)

## My Implementation

In [8]:
mlp = MLP()
model = LogitsToPredicate(mlp)

from ltn_imp.fuzzy_operators.aggregators import SatAgg
from ltn_imp.parsing.parser import convert_to_ltn

sat_agg_op = SatAgg()


In [9]:
predicates = {"Classifier": model }

expression = "all x. (Classifier(x,y))"
rule = convert_to_ltn(expression, predicates=predicates, functions=None, quantifier_impls={"forall" : "pmean_error"})

In [10]:
one =  torch.tensor([1, 0, 0])
two = torch.tensor([0, 1, 0])
three = torch.tensor([0, 0, 1])

In [11]:
# it computes the overall satisfaction level on the knowledge base using the given data loader (train or test)
def compute_sat_level(loader):
    mean_sat = 0
    for data, labels in loader:
        
        class_1 = data[labels == 0]
        class_2= data[labels == 1]
        class_3 = data[labels == 2]

        mean_sat += sat_agg_op(
            rule({"x": class_1, "y": one}),
            rule({"x": class_2, "y": two}),
            rule({"x": class_3, "y": three})
        )

    mean_sat /= len(loader)
    return mean_sat

# it computes the overall accuracy of the predictions of the trained model using the given data loader
# (train or test)
def compute_accuracy(loader):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = model.logits_model(data).detach().numpy()
        predictions = np.argmax(predictions, axis=1)
        mean_accuracy += accuracy_score(labels, predictions)

    return mean_accuracy / len(loader)

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(501):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        class_1 = data[labels == 0]
        class_2= data[labels == 1]
        class_3 = data[labels == 2]

        sat_agg = sat_agg_op(
            rule({"x": class_1, "y": one}),
            rule({"x": class_2, "y": two}),
            rule({"x": class_3, "y": three})
        )
        
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    if epoch % 100 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))

 epoch 0 | loss 0.6607 | Train Sat 0.343 | Test Sat 0.342 | Train Acc 0.349 | Test Acc 0.267
 epoch 100 | loss 0.1672 | Train Sat 0.836 | Test Sat 0.848 | Train Acc 0.965 | Test Acc 0.967
 epoch 200 | loss 0.1284 | Train Sat 0.874 | Test Sat 0.870 | Train Acc 0.983 | Test Acc 0.967
 epoch 300 | loss 0.0915 | Train Sat 0.896 | Test Sat 0.876 | Train Acc 0.983 | Test Acc 0.967
 epoch 400 | loss 0.1038 | Train Sat 0.896 | Test Sat 0.873 | Train Acc 0.983 | Test Acc 0.967
 epoch 500 | loss 0.1043 | Train Sat 0.905 | Test Sat 0.868 | Train Acc 0.992 | Test Acc 0.933


## LTN

In [13]:
l_A = ltn.Constant(torch.tensor([1, 0, 0]))
l_B = ltn.Constant(torch.tensor([0, 1, 0]))
l_C = ltn.Constant(torch.tensor([0, 0, 1]))

In [14]:
mlp = MLP()
P = ltn.Predicate(LogitsToPredicate(mlp))
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()

In [15]:
# it computes the overall satisfaction level on the knowledge base using the given data loader (train or test)
def compute_sat_level(loader):
    mean_sat = 0
    for data, labels in loader:
        x_A = ltn.Variable("x_A", data[labels == 0])
        x_B = ltn.Variable("x_B", data[labels == 1])
        x_C = ltn.Variable("x_C", data[labels == 2])
        mean_sat += SatAgg(
            Forall(x_A, P(x_A, l_A)),
            Forall(x_B, P(x_B, l_B)),
            Forall(x_C, P(x_C, l_C))
        )
    mean_sat /= len(loader)
    return mean_sat

# it computes the overall accuracy of the predictions of the trained model using the given data loader
# (train or test)
def compute_accuracy(loader):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = mlp(data).detach().numpy()
        predictions = np.argmax(predictions, axis=1)
        mean_accuracy += accuracy_score(labels, predictions)

    return mean_accuracy / len(loader)

In [16]:
optimizer = torch.optim.Adam(P.parameters(), lr=0.001)

for epoch in range(501):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        x_A = ltn.Variable("x_A", data[labels == 0]) # class A examples
        x_B = ltn.Variable("x_B", data[labels == 1]) # class B examples
        x_C = ltn.Variable("x_C", data[labels == 2]) # class C examples

        sat_agg = SatAgg(
            Forall(x_A, P(x_A, l_A, training=True)),
            Forall(x_B, P(x_B, l_B, training=True)),
            Forall(x_C, P(x_C, l_C, training=True))
        )

        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    if epoch % 100 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))

 epoch 0 | loss 0.6555 | Train Sat 0.348 | Test Sat 0.348 | Train Acc 0.648 | Test Acc 0.733
 epoch 100 | loss 0.3013 | Train Sat 0.758 | Test Sat 0.766 | Train Acc 0.982 | Test Acc 0.967
 epoch 200 | loss 0.2051 | Train Sat 0.860 | Test Sat 0.867 | Train Acc 0.984 | Test Acc 0.967
 epoch 300 | loss 0.1751 | Train Sat 0.877 | Test Sat 0.867 | Train Acc 0.991 | Test Acc 0.933
 epoch 400 | loss 0.1484 | Train Sat 0.905 | Test Sat 0.871 | Train Acc 0.984 | Test Acc 0.967
 epoch 500 | loss 0.1021 | Train Sat 0.901 | Test Sat 0.868 | Train Acc 0.991 | Test Acc 0.967
